### Neural Network architectures for feature extraction
This notebook shows an example of how to extract deep features by leveraging the architecture of Deep Neural Networks. 
The dataset used is MNIST - A database of handwritten digits 
#### Dataset Description
1. The MNIST dataset contains 60,000 Handwritten digits as training samples and 10,000 Test samples, 
which means each digit occurs 6000 times in the training set and 1000 times in the testing set. (approximately). 
2. Each image is Size Normalized and Centered 
3. Each image is 28 X 28 Pixel with 0-255 Gray Scale Value. 
4. That means each image is represented as 784 (28 X28) dimension vector where each value is in the range 0- 255.

#### Importing the required libraries

In [1]:
# import libraries
import torch
import numpy as np
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import lr_scheduler
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy

import warnings
warnings.filterwarnings("ignore")

c:\Users\Paul\anaconda3\envs\git_dl_experiments\lib\site-packages\torchvision\io\image.py:13: UserWarning: Failed to load image Python extension: [WinError 127] The specified procedure could not be found
  warn(f"Failed to load image Python extension: {e}")


In [2]:
# convert data to torch.FloatTensor
transform = transforms.Compose([transforms.ToTensor()])
# Choose the training and test datasets
train_data = datasets.MNIST(root='data', train=True,
                                   download=True, transform=transform)
test_data = datasets.MNIST(root='data', train=False,
                                  download=True, transform=transform)

AttributeError: module 'torch' has no attribute '_six'

In [3]:
# Initializing batch size
batch_size = 32

# Loading the train dataset
train_loader = torch.utils.data.DataLoader(dataset=train_data, 
                                           batch_size=batch_size, 
                                           shuffle=True)

# Loading the test dataset
test_loader = torch.utils.data.DataLoader(dataset=test_data, 
                                          batch_size=batch_size, 
                                          shuffle=True)

In [4]:
for (X_train, y_train) in train_loader:
    print('X_train:', X_train.size(), 'type:', X_train.type())
    print('y_train:', y_train.size(), 'type:', y_train.type())
    break

X_train: torch.Size([32, 1, 28, 28]) type: torch.FloatTensor
y_train: torch.Size([32]) type: torch.LongTensor


In [5]:
class Net_pretrained(nn.Module):
    def __init__(self):
        super(Net_pretrained, self).__init__()
        # linear layer (784 -> 1 hidden node)
        self.fc1 = nn.Linear(28 * 28, 512) # First fully connected layer which takes input image 28x28 --> 784
        self.fc2 = nn.Linear(512, 512)
        self.fc3 = nn.Linear(512, 512)
        self.fc4 = nn.Linear(512, 512)
        self.fc5 = nn.Linear(512, 10) # Last fully connected layer which outputs our 10 labels

    def forward(self, x):
        # The view function is meant to flatten the tensor (28x28 is converted to 784)  
        x = x.view(-1, 28 * 28)
        # Add hidden layer, with relu activation function
        # Relu an activation function which allows positive values to pass through the network, whereas negative values are modified to zero
        x1 = F.relu(self.fc1(x))
        x2 = F.relu(self.fc2(x1))
        x3 = F.relu(self.fc3(x2))
        x4 = F.relu(self.fc4(x3))
        output = self.fc5(x4)
        return output, x4

In [6]:
# use_cuda = torch.cuda.is_available()
# device = torch.device("cuda" if use_cuda else "cpu")
device = "cpu"
model_pretrained = Net_pretrained()
model_pretrained = model_pretrained.to(device) 
print(model_pretrained)

Net_pretrained(
  (fc1): Linear(in_features=784, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=512, bias=True)
  (fc3): Linear(in_features=512, out_features=512, bias=True)
  (fc4): Linear(in_features=512, out_features=512, bias=True)
  (fc5): Linear(in_features=512, out_features=10, bias=True)
)


In [7]:
learning_rate = 0.01

# specify loss function
criterion = nn.CrossEntropyLoss()

# specify optimizer
optimizer = torch.optim.SGD(model_pretrained.parameters(), lr=learning_rate)

In [8]:
def train(epoch, log_interval=100):
    # First switch the module mode to model.train() so that new weights can be learned after every epoch. 
    model_pretrained.train()

    # Loop through each batch of images in train set
    for batch_idx, (data, target) in enumerate(train_loader):
       
        data, target = data.to(device), target.to(device)

        # Zero out the gradients from the preivous step 
        optimizer.zero_grad()

        # Forward pass (this calls the "forward" function within Net)
        output, _ = model_pretrained(data)

        # Compute the Loss
        loss = criterion(output, target)

        # Do backward pass
        loss.backward()

        # optimizer.step() updates the weights accordingly
        optimizer.step()

        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

In [9]:
def test(loss_vector, accuracy_vector):
    model_pretrained.eval()                           # model.eval() here sets the PyTorch module to evaluation mode. 
                                           
    test_loss, correct = 0, 0

    for data, target in test_loader:
        data, target = data.to(device), target.to(device)  # Convert the data and target to Pytorch tensor 

        # Passing images/data to the model, which return the probabilites as outputs
        output,_ = model_pretrained(data) 

        # calculate the loss
        test_loss += criterion(output, target).item()

        # convert output with maximum probabilities to predicted class
        # # get the index of the max log-probability
        _, pred = torch.max(output, 1)

        # compare predictions to true label
        correct += (pred == target).sum().item()
    
    # Calculating the loss
    test_loss /= len(test_loader)
    loss_vector.append(test_loss)

    # Calculating the accuracy
    accuracy = 100. * correct / len(test_loader.dataset)

    accuracy_vector.append(accuracy)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset), accuracy))
    return accuracy_vector

In [10]:
%%time
epochs = 3

lossv, accv = [], []
acc_vector = []
for epoch in range(1, epochs + 1):
    train(epoch)
    acc_vector = test(lossv, accv)

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.298703
Train Epoch: 1 [3200/60000 (5%)]	Loss: 2.304837
Train Epoch: 1 [6400/60000 (11%)]	Loss: 2.294628
Train Epoch: 1 [9600/60000 (16%)]	Loss: 2.290450
Train Epoch: 1 [12800/60000 (21%)]	Loss: 2.290012
Train Epoch: 1 [16000/60000 (27%)]	Loss: 2.294807
Train Epoch: 1 [19200/60000 (32%)]	Loss: 2.286156
Train Epoch: 1 [22400/60000 (37%)]	Loss: 2.292574
Train Epoch: 1 [25600/60000 (43%)]	Loss: 2.271195
Train Epoch: 1 [28800/60000 (48%)]	Loss: 2.279348
Train Epoch: 1 [32000/60000 (53%)]	Loss: 2.265738
Train Epoch: 1 [35200/60000 (59%)]	Loss: 2.231220
Train Epoch: 1 [38400/60000 (64%)]	Loss: 2.235354
Train Epoch: 1 [41600/60000 (69%)]	Loss: 2.188515
Train Epoch: 1 [44800/60000 (75%)]	Loss: 2.097106
Train Epoch: 1 [48000/60000 (80%)]	Loss: 1.824871
Train Epoch: 1 [51200/60000 (85%)]	Loss: 1.745570
Train Epoch: 1 [54400/60000 (91%)]	Loss: 1.368737
Train Epoch: 1 [57600/60000 (96%)]	Loss: 1.036821

Test set: Average loss: 0.9339, Accuracy: 7179/10000 (72%)

In [11]:
# Specify a path
PATH = "network_pretrained_layer4_512n.pt"

# Save the pytorch trained model
torch.save(model_pretrained.state_dict(), PATH)

In [12]:
model_pretrained = Net_pretrained()
model_pretrained.load_state_dict(torch.load(PATH))
model_pretrained.eval()

Net_pretrained(
  (fc1): Linear(in_features=784, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=512, bias=True)
  (fc3): Linear(in_features=512, out_features=512, bias=True)
  (fc4): Linear(in_features=512, out_features=512, bias=True)
  (fc5): Linear(in_features=512, out_features=10, bias=True)
)

False